In [1]:
import os
import sys
print(os.getcwd())
import torch as th
from src.RL.Networks.ExtractorNetworks import ObjectTactileEncoder
from src.RL.control_dropping_env import BerrettHandGym

DATA_SAVE_PATH = os.path.join(os.getcwd(), '..', 'Data_Collection')


env = BerrettHandGym(detailed_training=True)
state_space = env.observation_space

/media/rpal/Drive_10TB/John/Control Drop


ModuleNotFoundError: No module named 'src'

In [ ]:
import os
import numpy as np
import pickle

def save_data(data, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))
    
    # Extract data from each data point and create numpy arrays
    palm_tactile_list = []
    finger_1_tactile_list = []
    finger_2_tactile_list = []
    finger_3_tactile_list = []
    palm_location_list = []
    finger_1_location_list = []
    finger_2_location_list = []
    finger_3_location_list = []
    obj_location_list = []
    obj_velocity_list = []
    action_list = []
    new_object_location_list = []
    
    for data_point in data:
        state, obj_location_vector = data_point
        palm_tactile_list.append(state['palm_tactile'])
        finger_1_tactile_list.append(state['finger_1_tactile'])
        finger_2_tactile_list.append(state['finger_2_tactile'])
        finger_3_tactile_list.append(state['finger_3_tactile'])
        palm_location_list.append(state['palm_location'])
        finger_1_location_list.append(state['finger_1_location'])
        finger_2_location_list.append(state['finger_2_location'])
        finger_3_location_list.append(state['finger_3_location'])
        obj_location_list.append(state['obj_location'].flatten())
        obj_velocity_list.append(state['obj_velocity'].flatten())
        action_list.append(state['action'])
        new_object_location_list.append(obj_location_vector.flatten())

    # Create a single numpy array for each component
    palm_tactile_array = np.array(palm_tactile_list).squeeze(axis=1)
    finger_1_tactile_array = np.array(finger_1_tactile_list).squeeze(axis=1)
    finger_2_tactile_array = np.array(finger_2_tactile_list).squeeze(axis=1)
    finger_3_tactile_array = np.array(finger_3_tactile_list).squeeze(axis=1)
    palm_location_array = np.array(palm_location_list)
    finger_1_location_array = np.array(finger_1_location_list)
    finger_2_location_array = np.array(finger_2_location_list)
    finger_3_location_array = np.array(finger_3_location_list)
    obj_location_array = np.array(obj_location_list)
    obj_velocity_array = np.array(obj_velocity_list)
    action_array = np.array(action_list)
    new_object_location_array = np.array(new_object_location_list)

    # Concate all arrays along axis 1
    data_array = np.concatenate([
        palm_tactile_array, finger_1_tactile_array, finger_2_tactile_array, finger_3_tactile_array,
        palm_location_array, finger_1_location_array, finger_2_location_array, finger_3_location_array,
        obj_location_array, obj_velocity_array, action_array, new_object_location_array
    ], axis=1)

    # Save the data array to file
    if os.path.exists(os.path.join(file_path, 'Actions_data.npy')):
        old_data = np.load(os.path.jostart_lr= 0.003
end_lr = 0.0001
factor = 0.999
def lr_schedule():
    global start_lr, end_lr, factor
    ret_lr = start_lr
    start_lr *= factor
    return ret_lrshape), mult(state_space['finger_1_tactile'].shape), mult(state_space['finger_2_tactile'].shape), mult(state_space['finger_3_tactile'].shape), mult(state_space['palm_location'].shape), mult(state_space['finger_1_location'].shape), mult(state_space['finger_2_location'].shape), mult(state_space['finger_3_location'].shape), mult(state_space['obj_location'].shape), mult(state_space['obj_velocity'].shape), 5, ]

    indexes = np.zeros(len(sizes), dtype=int)
    for i in range(len(sizes)):
        indexes[i] = sizes[i] + indexes[i-1] if i > 0 else sizes[i]
    palm_tactile_array, finger_1_tactile_array, finger_2_tactile_array, finger_3_tactile_array, \
    palm_location_array, finger_1_location_array, finger_2_location_array, finger_3_location_array, \
    obj_location_array, obj_velocity_array, action_array, new_object_location_array = np.split(data_array, indexes, axis=1)

    # Create a list of data points from the components
    data = []
    num_samples = data_array.shape[0]
    for i in range(num_samples):
        state = {
            'palm_tactile': palm_tactile_array[i].reshape((1, -1)),
            'finger_1_tactile': finger_1_tactile_array[i].reshape((1, -1)),
            'finger_2_tactile': finger_2_tactile_array[i].reshape((1, -1)),
            'finger_3_tactile': finger_3_tactile_array[i].reshape((1, -1)),
            'palm_location': palm_location_array[i],
            'finger_1_location': finger_1_location_array[i],
            'finger_2_location': finger_2_location_array[i],
            'finger_3_location': finger_3_location_array[i],
            'obj_location': obj_location_array[i].reshape((state_space['obj_location'].shape[0], state_space['obj_location'].shape[1])),
            'obj_velocity': obj_velocity_array[i].reshape((state_space['obj_velocity'].shape[0], state_space['obj_velocity'].shape[1])),
            'action': action_array[i],
        }
        obj_location_vector = new_object_location_array[i]
        data.append((state, obj_location_vector))

    return data


In [ ]:
load_data(os.path.join(os.getcwd(), 'Data_Collection', 'Action_Samples', 'Actions_data.npy'))

In [ ]:
# Dataset Collection

def DataCollect(num_steps):
    done = False
    data = []
    data_point = env.reset()[0].copy()
    i = 0
    while i < num_steps:
        if done:
            save_data(data, os.path.join(os.getcwd(), 'Data_Collection', 'Action_Samples'))
            data = []
            data_point = env.reset()[0].copy()
        i+=1
        action = env.action_space.sample()
        print('Action:', action)
        data_point['action'] = action
        state, reward, done, _ = env.step(action)
        ball_locations = env.simController.get_object_locations()
        print('Ball locations:', ball_locations)
        data.append((data_point, np.array(ball_locations)))
        data_point = state.copy()
    
DataCollect(50000)

In [ ]:
start_lr= 0.003
end_lr = 0.0001
factor = 0.999
def lr_schedule():
    global start_lr, end_lr, factor
    ret_lr = start_lr
    start_lr *= factor
    return ret_lr

In [ ]:
# Training the Keyword Model!!
from torch.utils.data.dataset import Dataset, random_split
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
import os
def train_model(
    model, train_loader, val_loader,
    batch_size = 16, 
    epochs = 1000, 
    learning_rate = 5e-3, 
    log_interval = 50, 
    no_cuda = False, 
    seed = 1, 
    is_lstm=False,
    patience = 10):

  model.set_batch_size(batch_size)
  print(model._batch_size)
  use_cuda = not no_cuda and th.cuda.is_available()
  device = th.device("cuda" if use_cuda else "cpu")
  print(device)
  kwargs = {}
  criterion = nn.CrossEntropyLoss()
  noise_level = 0.001  

  def train(model, device, train_loader, optimizer, is_lstm=is_lstm):
    model.train()
    model.to(device)
    total_loss = 0
    i = 0
    for data, target in train_loader:
        th.cuda.empty_cache()
        i+=1
        if is_lstm: model.reset_hidden_state(data.shape[0])
        # print('data', data.shape)
        data = data.to(device).float()
        target = target.to(device)
        optimizer.zero_grad()
        output_target = target.float()
        output_prediction = model(data)
        _, target_indices = output_target.max(dim=2)
        loss = 0
        for t in range(output_prediction.size(1)):
            loss += criterion(output_prediction[:, t], target_indices[:, t]) * batch_size
        loss /= output_prediction.size(1)
        loss.backward()
        optimizer.step()
        lr_schedule()
        total_loss+=loss.item()
        if i % log_interval == 0:
            try:
                print(f'Avg Loss: {(total_loss/i+1)}%')
            except:
                pass
    return total_loss / len(train_loader.dataset)

  def validation(model, device, val_loader, is_lstm=is_lstm):
    model.eval()
    loss_total = 0
    with th.no_grad():
      for data, target in val_loader:
        if is_lstm: model.reset_hidden_state(data.shape[0])
        data = data.to(device).float()
        target = target.to(device)
        output_target = target.float()
        output_prediction = model(data)
        _, target_indices = output_target.max(dim=2)
        val_loss = 0
        for t in range(output_prediction.size(1)):
            val_loss += criterion(output_prediction[:, t], target_indices[:, t]) * batch_size
        val_loss /= output_prediction.size(1)
        loss_total += val_loss.item()

    val_loss = loss_total / len(val_loader.dataset)
    print('Validation_loss:', val_loss)
    return val_loss

  model.to(device)

  optimizer = optim.Adam(model.parameters(), lr = start_lr,
        eps=1e-7,
        weight_decay=0.005,
        # momentum=0.92,
        # centered=True
    )
  print('Training...')
  for epoch in range(1, epochs+1):
    train_loss = train(model, device, train_loader, optimizer)
    if epoch % 10 == 0 :
        val_loss = validation(model, device, val_loader)
    if epoch % 50 == 0:
        model.save_checkpoint(os.path.join(os.getcwd(),'..','..', 'data', 'Models', 'KeywordModel', 'Training', 'Checkpoints', f'LSTMKeywordCheckpoint_{epoch}.zip'))
